## Warehouse Location problem
Olivier Rubel, PhD

Associate Professor of Marketing

UC Davis

Analytical Decision Making -- MSBA Program


Date: April, 2018

# Description of the Problem

You have to decide where to open warehouses to serve a customers located in four possible locations. The three possible locations are Harlingen, Memphis and Ashland. The four customer locations are NYC, LA, Chitown and Houston.

The distance (cost) between the candidate locations and customer locations are given in the following table

|  | NYC | LA | Chicago | Houston         
| :---:|:---: | :---:|:---:|:---:
| Harlingen|1956|1606|1410|330
| Memphis|1096|1792|531|567
| Ashland|485|2322|324|1236

You have the decide where to open warehouses such that to minimize the cost of shipping while respecting some constraints. For instance, all customers must be served and you cannot open more than two warehouses. 

Questions:
0. How many warehouses do you open?
1. Where are you going to location your warehouse(s)?
2. Who ships to whom?
3. What is the optimum?

## Modeling
This problem is known as the p-median problem, see for instance https://optimization.mccormick.northwestern.edu/index.php/Facility_location_problems

The mathematical formulation of the problem is as follows. Let $x_{n,m}$ be the fraction of demand for customer $m$ that is served by warehouse $n$. For each warehouse $n$, the cost of delivering the product to customer $m$ is given by $d_{n,m}$. The binary variables $y_n$ are used to define whether or not a warehouse should be built, where $y_n=1$ when wharehouse $n$ is open.

The objective function is thus:
\begin{equation}
\text{min}_{x,y}\quad \sum_{n=1}^3\sum_{m=1}^{4}  d_{n,m}\times x_{n,m}
\end{equation}
s.t.
\begin{align}
\sum_{n=1}^{3}x_{n,m}&= 1\quad \forall m\\
x_{n,m}& \leq y_n\quad \forall n,m\\
\sum_{n=1}^{3} y_n&\leq 2\\
0\leq x&\leq 1&\\
y\in&\{0,1\}&\\
\end{align}

# Solution in CVXPY


In [3]:
import numpy as np
import math
import cvxpy as cvx
from numpy import matrix 
from cvxpy import *
import pandas as pd

N=3 # Number of possible warehouses
M=4 # Number of destination
d=matrix([[1956,1606,1410,330],[1096,1792,531,567],[485,2322,324,1236]]) # cost (distance) matrix

# Define variables
x0 = cvx.Variable(M) # how much I ship from location0(Harlingen) to the 4 customer locations.
x1 = cvx.Variable(M)
x2 = cvx.Variable(M)
y = cvx.Bool(3) # define if the 3 warehouses are going to be built or not

Z0 = sum_entries(d[0]*x0)
Z1 = sum_entries(d[1]*x1)
Z2 = sum_entries(d[2]*x2)

# Construction of the Objective
objective = cvx.Minimize(Z0+Z1+Z2)

#Construction of the Constraints
## All customers must be served (represent each element in Xs as a proportion)
c01=x0[0] + x1[0] + x2[0] == 1 # demand from NYC fully satisfied
c02=x0[1] + x1[1] + x2[1] == 1 # demand from LA fully satisfied
c03=x0[2] + x1[2] + x2[2] == 1 # demand from Chicago fully satisfied
c04=x0[3] + x1[3] + x2[3] == 1 # demand from Houston fully satisfied

## Selected warehouse can deliver only if that warehouse is selected to be built
c40 = x0[0] <= y[0]
c41 = x0[1] <= y[0]
c42 = x0[2] <= y[0]
c43 = x0[3] <= y[0]

c50 = x1[0] <= y[1]
c51 = x1[1] <= y[1]
c52 = x1[2] <= y[1]
c53 = x1[3] <= y[1]

c60 = x2[0] <= y[2]
c61 = x2[1] <= y[2]
c62 = x2[2] <= y[2]
c63 = x2[3] <= y[2]

##Number of warehouse to build is limited to 2
c7 = y[0] + y[1] + y[2] <= 2

##xs must be between zero and one
c80 = x0 <= 1
c81 = x0 >= 0
c90 = x1 <= 1
c91 = x1 >= 0
c100 = x2 <= 1
c101 = x2 >= 0

## set of constraints
con=[c01, c02, c03, c04, c40, c41, c42, c43, c50, c51, c52, c53, c60, c61, c62, c63, c7, c80, c81, c90, c91, c100, c101]

prob = cvx.Problem(objective, con)
result = prob.solve()

print('The minimum cost is:')
print(prob.value)
print('Warehouses to open are:')
print(y.value)
print('From H:')
print(x0.value)
print('From M:')
print(x1.value)
print('From A:')
print(x2.value)

The minimum cost is:
2745.0000002231086
Warehouses to open are:
[[  1.00000000e+00]
 [  2.89323802e-10]
 [  1.00000000e+00]]
From H:
[[  7.48606466e-12]
 [  1.00000000e+00]
 [  7.62663462e-12]
 [  1.00000000e+00]]
From M:
[[  9.00523053e-11]
 [  2.40461225e-10]
 [  1.02645314e-10]
 [  2.44278926e-10]]
From A:
[[  1.00000000e+00]
 [  1.81199268e-11]
 [  1.00000000e+00]
 [  1.27202881e-11]]


## Solution

The cost at optimum is 2745.

Harlinger and Ashland are opened. Harlinger serves LA and Hosuton. Ashland serves NYC and Chitowm


